In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold

save_base = "../preprocessed_data/"


In [49]:
class SensorDataset(Dataset):
    def __init__(self):
        # UV_x = np.loadtxt(save_base + "UV_x_scaled.csv", delimiter=',')
        # VIS_x = np.loadtxt(save_base + "VIS_x_scaled.csv", delimiter=',')
        IR_x = np.loadtxt(save_base + "IR_x_scaled.csv", delimiter=',')
        y = np.loadtxt(save_base + "y.csv", delimiter=',')
        
        self.X = torch.from_numpy(IR_x).float()
        self.y = torch.from_numpy(y).float().view(-1, 1)
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, ix):
        x = self.X[ix]
        y = self.y[ix]
        return x, y
    
    def get_dim(self):
        return self.X.shape[1]
    
dataset = SensorDataset()

In [52]:
hidden1 = 128
hidden2 = 64

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(dataset.get_dim(), hidden1),
            nn.Tanh(),
            nn.Linear(hidden1, hidden2),
            nn.Tanh(),
            nn.Linear(hidden2, 1)
        )

    def forward(self, x):
        return self.layers(x)
    
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    layer.reset_parameters()

In [53]:
k_folds = 7
epochs = 800

kf = KFold(n_splits=k_folds, shuffle=True)
loss_function = nn.MSELoss()

fold_train_losses = []
fold_test_losses = []

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kf.split(dataset)):

  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = SubsetRandomSampler(train_ids)
  test_subsampler = SubsetRandomSampler(test_ids)

  # Define data loaders for training and testing data in this fold
  trainloader = DataLoader(
                    dataset, 
                    batch_size=32, sampler=train_subsampler)
  testloader = DataLoader(
                    dataset,
                    batch_size=32, sampler=test_subsampler)

  # Init the neural network
  network = MLP()
  network.apply(reset_weights)

  # Initialize optimizer
  optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
  
  train_loss = 0

  # Run the training loop for defined number of epochs
  for epoch in range(0, epochs):

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get inputs
      inputs, targets = data
      
      # Zero the gradients
      optimizer.zero_grad()
      
      # Perform forward pass
      outputs = network(inputs)
      
      # Compute loss
      loss = loss_function(outputs, targets)
      
      # Perform backward pass
      loss.backward()
      
      # Perform optimization
      optimizer.step()
      
      current_loss += torch.sqrt(loss).item()
    
    current_loss /= len(trainloader)
    train_loss = current_loss
    
  fold_train_losses.append(train_loss)
  
  print(f'Fold {fold} train loss: {train_loss}')

  with torch.no_grad():
    test_loss = 0
    
    for i, data in enumerate(testloader, 0):
      
      inputs, targets = data
      
      outputs = network(inputs)
      
      loss = loss_function(outputs, targets)
      
      test_loss += torch.sqrt(loss).item()
      
    test_loss /= len(testloader)
    
    fold_test_losses.append(test_loss)
    
    print(f'Fold {fold} test loss: {test_loss}')
    print("---------------------------")
        
print(f'Mean train loss: {np.mean(fold_train_losses)}')
print(f'Mean test loss: {np.mean(fold_test_losses)}')
      

Fold 0 train loss: 0.7993678450584412
Fold 0 test loss: 2.2861573696136475
---------------------------
Fold 1 train loss: 0.9893915057182312
Fold 1 test loss: 0.8533369898796082
---------------------------
Fold 2 train loss: 0.9003667831420898
Fold 2 test loss: 1.7036198377609253
---------------------------
Fold 3 train loss: 0.9776577949523926
Fold 3 test loss: 0.8189162015914917
---------------------------
Fold 4 train loss: 0.9524540305137634
Fold 4 test loss: 1.0322901010513306
---------------------------
Fold 5 train loss: 0.9675295352935791
Fold 5 test loss: 0.5597501397132874
---------------------------
Fold 6 train loss: 0.87746262550354
Fold 6 test loss: 1.4769041538238525
---------------------------
Mean train loss: 0.9234614457402911
Mean test loss: 1.2472821133477348
